# 蒙特卡洛(Monte Carlo,MC)方法
对应《强化学习的数学原理》第5章

## 环境

In [2]:
import numpy as np
class CliffWalkingEnv:
    """ 悬崖漫步环境"""
    def __init__(self, ncol=4, nrow=4):   #4行4列
        self.ncol = ncol  # 定义网格世界的列
        self.nrow = nrow  # 定义网格世界的行
        self.P = self.createP() # 转移矩阵P[state][action] = [(p, next_state, reward, done)]包含下一个状态和奖励

    def createP(self):
        # 初始化P,空的，3维
        P = [[[] for j in range(4)] for i in range(self.nrow * self.ncol)]
        # 4种动作, change[0]，即[0,-1]是向上走,change[1]:下, change[2]:左, change[3]:右。坐标系原点(0,0)
        # 定义在左上角，坐标系原点 (0,0) 定义在左上角，往右和下坐标值会变大
        change = [[0, -1], [0, 1], [-1, 0], [1, 0]]
        #[0,-1]中第一个数表示对列的变化，因为列的改变对应坐标轴的x
        for i in range(self.nrow):      #在哪一行
            for j in range(self.ncol):  #在哪一列
                for a in range(4):      #哪个动作
                    # 位置在悬崖或者目标状态,因为无法继续交互,任何动作奖励都为0
                    if i == self.nrow - 1 and j > 0:    #如果在悬崖或者终点，还是在原地呢
                        P[i * self.ncol + j][a] = [(1, i * self.ncol + j, 0, True)]
                        continue
                    # 其他位置 ，在（i,j）采取第a个动作
                    next_x = min(self.ncol - 1, max(0, j + change[a][0]))  # j是纵坐标，[0,-1],第一个数表示列，列不变
                    next_y = min(self.nrow - 1, max(0, i + change[a][1]))  #行
                    next_state = next_y * self.ncol + next_x
                    reward = -1
                    done = False
                    # 下一个位置在悬崖或者终点
                    if next_y == self.nrow - 1 and next_x > 0:
                        done = True
                        if next_x != self.ncol - 1:  # 下一个位置在悬崖
                            reward = -100
                    P[i * self.ncol + j][a] = [(1, next_state, reward, done)]
        return P

# MC-first visit

In [3]:
class MCAgent:
    def __init__(self, env, gamma=0.9, epsilon=0.1):
        self.env = env
        self.gamma = gamma
        self.epsilon = epsilon
        self.nA = 4                    #每个state的动作，4个
        self.nS = env.ncol * env.nrow  #总共的状态数
        self.Q = np.zeros((self.nS, self.nA))  #初始化动作值，每一行表示该状态的4个动作值，总共 状态数行，4列,二维的

        self.returns_sum = [[[] for _ in range(self.nA)] for _ in range(self.nS)]  # 3维空列表，用来记录每一个s,a的回报
        self.policy = np.ones((self.nS, self.nA)) / self.nA                        # 均匀策略初始化,全都是0.25

    def choose_action(self, state):   # 选择动作，有epsilon的概率随机选，1-epsilon的概率选policy中概率最大的动作
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.nA)
        else:
            a = self.policy[state]
            max_index = np.random.choice(np.flatnonzero(a == a.max()))
            return max_index   #返回该状态action value最大的索引，对应的就是动作

    def generate_episode(self, state,action,max_steps=100 ):  #根据当前策略生成一个完整的episode
        #需要设置起点，策略迭代不用设置起点
        episode = []
        steps = 0
        p, next_state, reward, done = self.env.P[state][action][0]
        episode.append((state,action,reward))
        state = next_state
        if done == True:
            return episode
        if done == False :  #知道走到悬崖或终点了才停止
            while True:
                choose_a = self.choose_action(state)
                p, next_state, reward, done = self.env.P[state][choose_a][0]
                episode.append((state, choose_a, reward))
                state = next_state
                steps += 1
                if done or steps >= max_steps:  #知道走到悬崖或终点了才停止
                    break
            return episode

    def update_Q(self, episode):
        """首次访问蒙特卡罗更新Q表"""
        G = 0
        visited = set()  #创建一个集合
        for t in reversed(range(len(episode))):  #反向，第一个t不是0，是最大的数
            s, a, r = episode[t]                 #所以这里取的是这个episode中最后一个(s,a,r)
            G = self.gamma * G + r               #G就是当前这个s,a的q值，准确来说是q的估计值
            if (s, a) not in visited:  #如果没有被访问过
                visited.add((s, a))
                #returns_sum[s][a]是一个一维列表，往里面添加回报G,然后求平均，就是作为这个q(s,a)
                self.returns_sum[s][a].append(G)  #把这个G放进去，这里面还包含之前的episode的这个s,a的G
                self.Q[s][a] = np.mean(self.returns_sum[s][a]) #然后求平均来近似为q

                # 更新策略
                best_a = np.argmax(self.Q[s])  #取出状态s下使得q最大的a,但是后面更新策略不是让这个a的概率为1，让其他动作也有一些小概率
                for action in range(self.nA):
                    if action == best_a: # 让q最的大action的概率在策略中是1-self.epsilon+self.epsilon / self.nA，其他的是epsilon / nA
                        self.policy[s][action] = 1 - (self.nA-1)/self.nA*self.epsilon # 1-3/4*ε
                    else:
                        self.policy[s][action] = self.epsilon / self.nA               # 1/4*ε


    def train(self,num_episodes_each=100):
        for s in range(0,13):  # 即从除了悬崖和终点以外的每个状态每个动作出发
            for a in range(self.nA):
                for _ in range(num_episodes_each):  # 每个(s,a)采样 N 次，在这个s,a开始采样，采N个。

                    episode = self.generate_episode(s, a)   #在这个s,a开始采第一个，返回一个episode,
                    self.update_Q(episode)    #用刚得的episode求return,但是这个过程可以得到很多qsa的估计值，都存起来求平均。等到了下一个episode,有新加入了一个值，在求平均更新
        return self

采样完一个episode之后，把这个episode过程中的s,a,r都记录下来，然后开始更新q表和policy
<br>反向依次取这个episode中的s,a,r,每反着取一个，就更新一次Q表和policy。
<br>但是这里是first visit,也就是说反着取的第一个s,a才更新，如果是又遇到了s,a,就不更新了
<br>所以说first visit使用的其实是这个episode中最后一次出现的s,a(比如这个s,a出现了2次，用的是后面那次出现的s,a)
<br>但是标准的首次访问蒙特卡罗应该是使用episode中第一次出现的s,a。而这里的代码用的是episode中的最后一次出现的s,a,这里代码可以改一下。

## 实例化

In [4]:
# 环境
env = CliffWalkingEnv(ncol=4, nrow=4)
# 算法
agent = MCAgent(env, gamma=0.9, epsilon=0.1)

## 训练

In [5]:
agent.train(num_episodes_each=600)

## 训练结束

### 打印q表

In [6]:
print("==== Q(s,a) 值表 ====")
for s in range(agent.nS):
    print(f"状态 {s}: {agent.Q[s]}")

==== Q(s,a) 值表 ====
状态 0: [-8.72239806 -8.04251944 -5.72424335 -7.51929959]
状态 1: [-5.67768376 -6.73399569 -8.61997994 -4.53386219]
状态 2: [-4.9046502  -4.34911064 -5.7610628  -3.73631936]
状态 3: [-3.80933347 -2.93304997 -4.42944464 -3.77006209]
状态 4: [-8.78504077 -7.42778108 -7.04413321 -4.91250991]
状态 5: [-6.94884485 -8.45952226 -6.21126349 -3.95917102]
状态 6: [-5.37934345 -5.35747925 -4.71726804 -2.87671141]
状态 7: [-3.73736726 -1.99329226 -3.94497441 -3.08451133]
状态 8: [ -5.65480556  -9.38463418  -6.06052929 -10.09082206]
状态 9: [  -6.49715199 -100.           -6.24991169   -5.43854359]
状态 10: [  -3.70099462 -100.           -8.32582457   -2.08226588]
状态 11: [-2.97924546 -1.         -5.50811173 -2.05306355]
状态 12: [  -5.95724463   -8.7946115    -9.01483904 -100.        ]
状态 13: [0. 0. 0. 0.]
状态 14: [0. 0. 0. 0.]
状态 15: [0. 0. 0. 0.]


### 策略可视化
注意，训练完成后，我获得了一个policy,一个q表。然后来测试。测试时不要再用epsilon-greedy了，我得直接选最好的动作。
<br>我是根据policy选概率最大的，还是根据q表选使得q最大的呢
<br>根据q表来选，而不是根据policy来选

In [7]:
action_meaning = ['^', 'v', '<', '>']
print("\n最优策略（箭头形式）")
best_actions = np.argmax(agent.Q, axis=1).reshape(env.nrow, env.ncol) #对于q表的每一行，取出其中最大的，得到其索引。然后把取出的动作的索引按照环境的几行几列重新排成2维np.array
for row in best_actions:
    print(' '.join([action_meaning[a] for a in row]))


最优策略（箭头形式）
< > > v
> > > v
^ > > v
^ ^ ^ ^


这里第三排的第一个位置怎么是向上走，为什么不是直接向右走？
<br>把环境中的reward = -100改成-10再试试

# MC-every visit
在MC-1的基础上，把update_Q由first visit改成every visit

In [8]:
class MCAgent2:
    def __init__(self, env, gamma=0.9, epsilon=0.1):
        self.env = env
        self.gamma = gamma
        self.epsilon = epsilon
        self.nA = 4                    #每个state的动作，4个
        self.nS = env.ncol * env.nrow  #总共的状态数
        self.Q = np.zeros((self.nS, self.nA))  #初始化动作值，每一行表示该状态的4个动作值，总共 状态数行，4列,二维的

        self.returns_sum = [[[] for _ in range(self.nA)] for _ in range(self.nS)]  # 3维空列表，用来记录每一个s,a的回报
        self.policy = np.ones((self.nS, self.nA)) / self.nA                        # 均匀策略初始化,全都是0.25

    def choose_action(self, state):   # 选择动作，有epsilon的概率随机选，1-epsilon的概率选policy中概率最大的动作
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.nA)
        else:
            a = self.policy[state]
            max_index = np.random.choice(np.flatnonzero(a == a.max()))
            return max_index   #返回该状态action value最大的索引，对应的就是动作

    def generate_episode(self, state,action,max_steps=100 ):  #根据当前策略生成一个完整的episode
        #需要设置起点，策略迭代不用设置起点
        episode = []
        steps = 0
        p, next_state, reward, done = self.env.P[state][action][0]
        episode.append((state,action,reward))
        state = next_state
        if done == True:
            return episode
        if done == False :  #知道走到悬崖或终点了才停止
            while True:
                choose_a = self.choose_action(state)
                p, next_state, reward, done = self.env.P[state][choose_a][0]
                episode.append((state, choose_a, reward))
                state = next_state
                steps += 1
                if done or steps >= max_steps:  #知道走到悬崖或终点了才停止
                    break
            return episode

    def update_Q(self, episode):   #使用every visit
        G = 0
        for t in reversed(range(len(episode))):
            s, a, r = episode[t]
            G = self.gamma * G + r
            self.returns_sum[s][a].append(G)               # 存进去一个G
            self.Q[s][a] = np.mean(self.returns_sum[s][a]) # 更新一次q表

            # 更新策略
            best_a = np.argmax(self.Q[s])  #取出状态s下使得q最大的a,但是后面更新策略不是让这个a的概率为1，让其他动作也有一些小概率
            for action in range(self.nA):
                if action == best_a: # 让q最的大action的概率在策略中是1-self.epsilon+self.epsilon / self.nA，其他的是epsilon / nA
                    self.policy[s][action] = 1 - (self.nA-1)/self.nA*self.epsilon # 1-3/4*ε
                else:
                    self.policy[s][action] = self.epsilon / self.nA               # 1/4*ε

    def train(self,num_episodes_each=100):
        for s in range(0,13):  # 即从除了悬崖和终点以外的每个状态每个动作出发
            for a in range(self.nA):
                for _ in range(num_episodes_each):  # 每个(s,a)采样 N 次，在这个s,a开始采样，采N个。

                    episode = self.generate_episode(s, a)   #在这个s,a开始采第一个，返回一个episode,
                    self.update_Q(episode)    #用刚得的episode求return,但是这个过程可以得到很多qsa的估计值，都存起来求平均。等到了下一个episode,有新加入了一个值，在求平均更新
        return self

采样完一个episode之后，把这个episode过程中的s,a,r都记录下来，然后开始更新q表和policy
<br>反向依次取这个episode中的s,a,r,每反着取一个，就更新一次Q表和policy。
<br>every visit就不涉及first visit中那个第一次还是最后一次的问题了。

## 实例化

In [9]:
# 环境
env2 = CliffWalkingEnv(ncol=4, nrow=4)
# 算法
agent2 = MCAgent2(env, gamma=0.9, epsilon=0.1)

## 训练

In [10]:
agent2.train(num_episodes_each=600)

## 训练结束

### 打印q表

In [11]:
print("==== Q(s,a) 值表 ====")
for s in range(agent2.nS):
    print(f"状态 {s}: {agent2.Q[s]}")

==== Q(s,a) 值表 ====
状态 0: [-6.24877745 -5.68333369 -8.10769083 -5.1072529 ]
状态 1: [-5.14604774 -4.69957616 -5.60186656 -4.42857767]
状态 2: [-4.83069777 -3.74233878 -4.97617256 -3.82563875]
状态 3: [-3.87246358 -2.90097621 -4.39493079 -4.39260906]
状态 4: [-5.6182458  -6.03655458 -7.53332124 -4.62359955]
状态 5: [-5.08325976 -7.4907769  -5.16396014 -3.85394567]
状态 6: [-4.37575594 -4.61422907 -4.76713474 -2.98354045]
状态 7: [-3.81611515 -2.07438356 -3.72470324 -3.33160305]
状态 8: [-5.20568789 -8.34009459 -6.31566791 -7.96089484]
状态 9: [  -4.54528535 -100.           -5.73064118   -6.50836008]
状态 10: [  -3.91076998 -100.           -8.07817087   -2.10297265]
状态 11: [-2.94294987 -1.         -5.27924937 -2.01530129]
状态 12: [  -5.73191116   -9.0367629   -10.16422892 -100.        ]
状态 13: [0. 0. 0. 0.]
状态 14: [0. 0. 0. 0.]
状态 15: [0. 0. 0. 0.]


### 策略可视化

In [12]:
action_meaning = ['^', 'v', '<', '>']
print("\n最优策略（箭头形式）")
best_actions_2 = np.argmax(agent2.Q, axis=1).reshape(env.nrow, env.ncol)
for row in best_actions_2:
    print(' '.join([action_meaning[a] for a in row]))


最优策略（箭头形式）
> > v v
> > > v
^ ^ > v
^ ^ ^ ^
